In [26]:
%load_ext autoreload


import base64
from dotenv import load_dotenv
from anthropic import Anthropic, AsyncAnthropic

load_dotenv()

client = Anthropic()
model = "claude-3-7-sonnet-latest"

client = Anthropic()


def generate(messages, tools):
    response = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=1000,
        temperature=0.7,
        messages=messages,
        stream=False,
        tools=tools,
        tool_choice={"type": "auto"},
    )
    return response


def cv2base64(image_path):
    with open(image_path, "rb") as ifp:
        b64_image_str = base64.b64encode(ifp.read()).decode("utf-8")
    return b64_image_str


class Messages:
    def __init__(self):
        self.messages = []

    def _build_message(self, role, type, content_block):
        if type == "text":
            self.messages.append(
                {"role": role, "content": [{"type": type, "text": content_block}]}
            )
            print(f"{role}:{content_block}")

        elif type == "image" or type == "image+text":
            content = []
            for part in content_block.split("#"):
                if part.endswith("jpg"):
                    content.append(
                        {
                            "type": "image",
                            "source": {
                                "type": "base64",
                                "media_type": "image/jpeg",
                                "data": cv2base64(part),
                            },
                        }
                    )
                else:
                    content.append({"type": "text", "text": part})
            self.messages.append({"role": role, "content": content})
            print(f"{role}:{content}")
        elif type == "block":
            self.messages.append({"role": role, "content": content_block})
            print(f"{role}:{content_block}")

    def add_user_message(self, type, content):
        self._build_message("user", type, content)

    def add_assistant_message(self, type, content):
        self._build_message("assistant", type, content)

    def get_messages(self):
        return self.messages


async_client = AsyncAnthropic()


async def get_async_response(messages):
    async with async_client.messages.stream(
        model="claude-sonnet-4-20250514",
        max_tokens=1000,
        temperature=0.7,
        messages=messages,
    ) as stream:
        async for text in stream.text_stream:
            print(text, end=" ", flush=True)
        response = await stream.get_final_message()
        return response.content[0].text

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import datetime
from anthropic.types import ToolParam


def get_current_datetime(format="%Y-%m-%d %H:%M:%S"):
    try:
        return datetime.datetime.now().strftime(format)
    except Exception:
        raise ValueError(f"Incorrect date format:{format}")


def set_reminder(content, datetime_str):
    return f"Reminder timestamp:{datetime_str}, content:{content}"


def add_duration(datetime_str, duration_in_seconds=10, format="%Y-%m-%d %H:%M:%S"):
    try:
        date = datetime.datetime.strptime(datetime_str, format)
        date = date + datetime.timedelta(seconds=duration_in_seconds)
        return date.strftime(format)
    except Exception as e:
        raise RuntimeError(f"ERROR: {e}")


ADD_DURATION_SCHEMA = ToolParam(
    {
        "name": "add_duration",
        "description": "Adds a duration in seconds to a datetime string and returns the new datetime",
        "input_schema": {
            "type": "object",
            "properties": {
                "datetime_str": {
                    "type": "string",
                    "description": "The input datetime string to add duration to",
                },
                "duration_in_seconds": {
                    "type": "integer",
                    "description": "The number of seconds to add to the datetime",
                    "default": 10,
                },
                "format": {
                    "type": "string",
                    "description": "The datetime format string for parsing and formatting",
                    "default": "%Y-%m-%d %H:%M:%S",
                },
            },
            "required": ["datetime_str"],
        },
    }
)
SET_REMINDER_SCHEMA = ToolParam(
    {
        "name": "set_reminder",
        "description": "Creates a reminder with specified content and datetime",
        "input_schema": {
            "type": "object",
            "properties": {
                "content": {
                    "type": "string",
                    "description": "The content/message for the reminder",
                },
                "datetime_str": {
                    "type": "string",
                    "description": "The datetime string for when the reminder should trigger",
                },
            },
            "required": ["content", "datetime_str"],
        },
    }
)

DATETIME_TOOL_SCHEMA = ToolParam(
    {
        "name": "get_current_datetime",
        "description": "Get the current date and time in a specified format. Useful for timestamps, scheduling, time-sensitive operations, or when the current date/time is needed for context.",
        "input_schema": {
            "type": "object",
            "properties": {
                "format": {
                    "type": "string",
                    "description": "Python strftime format string for datetime formatting. Common formats: '%Y-%m-%d %H:%M:%S' (2024-03-15 14:30:45), '%Y-%m-%d' (2024-03-15), '%B %d, %Y' (March 15, 2024), '%I:%M %p' (2:30 PM)",
                    "default": "%Y-%m-%d %H:%M:%S",
                },
            },
            "required": ["format"],
        },
    }
)


def tool_call_handler(tool_name, tool_input):
    if tool_name == "get_current_datetime":
        return get_current_datetime(**tool_input)
    if tool_name == "add_duration":
        return add_duration(**tool_input)
    if tool_name == "set_reminder":
        return set_reminder(**tool_input)

    return f"ERROR - Unknown tool:{tool_name}::{tool_input}"


def chat(messages: Messages, tools):
    while True:
        response = generate(messages.get_messages(), tools=tools)
        messages.add_assistant_message("block", response.content)
        if response.stop_reason == "tool_use":
            for block in response.content:
                if block.type == "tool_use":
                    tool_input = block.input
                    tool_name = block.name
                    tool_id = block.id
                    tool_response = tool_call_handler(tool_name, tool_input)
                    tool_result = {
                        "tool_use_id": tool_id,
                        "type": "tool_result",
                        "content": tool_response,
                        "is_error": tool_response.find("ERROR") != -1,
                    }
                    messages.add_user_message("block", [tool_result])
        else:
            response_text = "\n".join(
                [block.text for block in response.content if block.type == "text"]
            )
            messages.add_assistant_message("text", response_text)
            break

In [37]:
messages = Messages()
tools = [DATETIME_TOOL_SCHEMA, ADD_DURATION_SCHEMA, SET_REMINDER_SCHEMA]
messages.add_user_message(
    "text",
    "here are a few todos: pickup laundry in 1 hour, finish up agents course before tomorrow, drive to bellevue at 6",
)
chat(messages, tools)

user:here are a few todos: pickup laundry in 1 hour, finish up agents course before tomorrow, drive to bellevue at 6
assistant:[TextBlock(text="I'll help you set up reminders for these todos. Let me get the current time first and then create the reminders.", type='text'), ToolUseBlock(id='toolu_018af84CfJoSUQsdwPxbzFfR', input={'format': '%Y-%m-%d %H:%M:%S'}, name='get_current_datetime', type='tool_use')]
user:[{'tool_use_id': 'toolu_018af84CfJoSUQsdwPxbzFfR', 'type': 'tool_result', 'content': '2025-08-08 16:18:04', 'is_error': False}]
assistant:[TextBlock(text='Now let me set up your reminders:', type='text'), ToolUseBlock(id='toolu_01NZJFoFND2TiX5wYGkCLapJ', input={'datetime_str': '2025-08-08 16:18:04', 'duration_in_seconds': 3600, 'format': '%Y-%m-%d %H:%M:%S'}, name='add_duration', type='tool_use'), ToolUseBlock(id='toolu_01XENoTFVkA7QndnNj3k1qGT', input={'content': 'pickup laundry', 'datetime_str': '2025-08-08 23:59:59'}, name='set_reminder', type='tool_use'), ToolUseBlock(id='too

In [38]:
messages.get_messages()

[{'role': 'user',
  'content': [{'type': 'text',
    'text': 'here are a few todos: pickup laundry in 1 hour, finish up agents course before tomorrow, drive to bellevue at 6'}]},
 {'role': 'assistant',
  'content': [TextBlock(text="I'll help you set up reminders for these todos. Let me get the current time first and then create the reminders.", type='text'),
   ToolUseBlock(id='toolu_018af84CfJoSUQsdwPxbzFfR', input={'format': '%Y-%m-%d %H:%M:%S'}, name='get_current_datetime', type='tool_use')]},
 {'role': 'user',
  'content': [{'tool_use_id': 'toolu_018af84CfJoSUQsdwPxbzFfR',
    'type': 'tool_result',
    'content': '2025-08-08 16:18:04',
    'is_error': False}]},
 {'role': 'assistant',
  'content': [TextBlock(text='Now let me set up your reminders:', type='text'),
   ToolUseBlock(id='toolu_01NZJFoFND2TiX5wYGkCLapJ', input={'datetime_str': '2025-08-08 16:18:04', 'duration_in_seconds': 3600, 'format': '%Y-%m-%d %H:%M:%S'}, name='add_duration', type='tool_use'),
   ToolUseBlock(id='too